# 객체 참조, 가변성, 재활용

## 8.1 변수는 상자가 아니다
이 챕터는 교육자의 마음가짐과 자질에 대한....  
내용이 아닌 변수를 박스로 보지 말고 라벨로 보자는 말을 함.

In [1]:
#example 8-1
a = [1, 2, 3]
b = a
a.append(4)
b

[1, 2, 3, 4]

예제 8-1은 '박스로서의 변수'가 먹히지 않는 간단한 예제. '참조'를 사용하는 언어에서는 조심해야 하는 비유이다. 변수 이름은 실제 데이터에 붙여진 포스트잇 정도로 생각하면 될 거 같다.  
<blockquote>변수가 객체에 할당되었다.</blockquote>
객체는 할당 전에 생성.

In [2]:
#Example 8-2
class Gizmo:
    def __init__(self):
        print('Gizmo id: %d' % id(self))
        
x = Gizmo()
y = Gizmo() * 10

Gizmo id: 140526505106792
Gizmo id: 140526505106960


TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'

In [3]:
print(dir())

['Gizmo', 'In', 'Out', '_', '_1', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i2', '_i3', '_ih', '_ii', '_iii', '_oh', 'a', 'b', 'exit', 'get_ipython', 'quit', 'x']


`dir()`을 확인해보면 y는 만들어지지 않았는데 y에 할당하려고 했던 객체의 `__init__()` 함수는 실행되었다. 객체는 할당 전에 생성된다는 증거. 앞에서 말한 개념을 생각해보면 당연하다. 변수(variable)은 그저 라벨일 뿐이기 때문이다. 객체 하나에 포스트잇 여러 개 붙는 것은 가능하다!
## 8.2 정체성, 동질성, 별명

In [4]:
#Example 8-3
charles = {'name': 'Charles L. Dodgson', 'born': 1832}
lewis = charles
print(lewis is charles)
print(id(charles), id(lewis))
lewis['balance'] = 950
charles

True
140526524439936 140526524439936


{'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}

**사기꾼** 등장!(갑자기?)

In [5]:
#Example 8-4
alex = {'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}
print(alex == charles)
print(alex is not charles)

True
True


완전 띠용인 결과가 아닐 수 없다....! 이유를 알아보자면,  
`==`은 `dict` class 안에 구현되어 있는 `__eq__` 함수를 호출하기 때문에 내용이 같으면 동일하다고 판단하지만,  
실제로는 구별된 객체이다. `a is not b`는 동일하지 않은 것을 비교하는 파이썬스러운 방법이다.

예제 8-3은 aliasing(별명)의 예제. 두 개의 변수가 하나의 객체에 묶여있음. 하지만 (사기꾼) alex는 내용은 같지만 동일한 객체에 묶여있지는 않다. 3.1절로 잠깐 돌아가보면, `is` 연산자는 두 객체의 '근본'을 비교한다. 이 근본(identity)은 객체마다 한번 만들어지면 변하지 않는다. identity를 불러올 때 `id()`라는 함수를 쓰는데 이건 어디서 구현했냐에 따라서 달라진다고 합니다..

### 그래서 ==냐 is냐?
`==`은 객체의 값을 비교하고,  
`is`는 identity를 비교함.

값이 더 중요한 거면 `==`을,  
싱글톤...이라면...
**토막상식**  
싱글톤 패턴이란?: 클래스가 *최초 한번만* 메모리를 할당하고(static) 그 메모리에 인스턴스를 만들어서 사용하는 디자인 패턴. 고정된 메모리를 사용하면서 인스턴스를 사용하기 때문에 메모리를 절약할 수 있음.  
은 잘 모르겠어요..

여튼 싱글톤이라면 `is`를 사용합시다. 지금까지 가장 많이 사용되는 예는 `None`인지 검사하는 용도.

#### 속도는..?
`is`가 오버로딩도 안되고 훨씬 빠르다. 그냥 두개의 ID만 비교하면 되기 때문!  
`object`에서 `==`은 `is`와 같지만, 대부분은 상속하면서 오버라이딩 하기 때문에 느려질 수밖에 없다.

### The relative immutability of Tuples
튜플도 다른 파이썬 자료구조들과 마찬가지로 객체의 '참조'를 가지고 있다. 하지만 앞에서도 잠깐 다뤘다시피, 튜플이 참조하는 객체의 mutability는 튜플과 무관하다. 다시 말하면, 튜플 자체에 들어있는 값(여기서는 참조값들)만 immutable이고 참조된 객체들은 신경쓰지 않는다!

In [6]:
#Example 8-5
t1 = (1, 2, [30, 40])
t2 = (1, 2, [30, 40])
print(t1 == t2)
print(id(t1[-1]))
t1[-1].append(99)
print(t1)
print(id(t1[-1]))
print(t1 == t2)

True
140526505346504
(1, 2, [30, 40, 99])
140526505346504
False


## 8.3 기본 복사는 얕은 복사

In [7]:
l1 = [3, [55, 44], (7, 8, 9)]
l2 = list(l1)
print(l2)
print(l2 == l1)
print(l2 is l1)

[3, [55, 44], (7, 8, 9)]
True
False


mutable sequence에 대해서는 `l2 = l1[:]`을 복사하는 단축어로 사용할 수 있다. 어쨌든 생성자나 [:]을 쓰는 것은 얕은 복사이다. 가장 바깥의 컨테이너는 복사되었지만 안에 내용물들은 원래 컨테이너의 참조들로 채워져있다는 말. 모든 아이템이 immutable하면 메모리도 아끼고 아무 문제가 없다. 하지만 mutable item이 있다면............^^

In [8]:
#Example 8-6
l1 = [3, [55, 44], (7, 8, 9)]
l2 = list(l1) #shallow copy
l1.append(100)
l1[1].remove(55)
print('l1: ', l1)
print('l2: ', l2)
l2[1] += [33, 22]
l2[2] += (10, 11)
print('l1:', l1)
print('l2:', l2)

l1:  [3, [44], (7, 8, 9), 100]
l2:  [3, [44], (7, 8, 9)]
l1: [3, [44, 33, 22], (7, 8, 9), 100]
l2: [3, [44, 33, 22], (7, 8, 9, 10, 11)]


설-명(지금까지 배웠던 내용들의 종합선물세트)
### 임의의 객체의 깊은 복사와 얕은 복사
항상 얕은 복사가 문제가 되는 건 아니지만, 그래도 깊은 복사가 필요할 때가 있다! 이럴 때는 `copy` 모듈에서 `deepcopy()`와 `copy()` 함수를 사용하자.

In [9]:
#Example 8-8
class Bus:
    
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
            
    def pick(self, name):
        self.passengers.append(name)
    
    def drop(self, name):
        self.passengers.remove(name)

In [10]:
#Example 8-9
import copy
bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)
print(id(bus1), id(bus2), id(bus3))

bus1.drop('Bill')
print(bus2.passengers)
print(id(bus1.passengers), id(bus2.passengers), id(bus3.passengers))
print(bus3.passengers)

140526504208984 140526504209040 140526504209264
['Alice', 'Claire', 'David']
140526505099400 140526505099400 140526505129992
['Alice', 'Bill', 'Claire', 'David']


*(대충 deep copy하면 참조가 아니라 지워도 안바뀐다는 예제)*

In [11]:
#Exaple 8-10
a = [10, 20]
b = [a, 30]
a.append(b)
print(a)

from copy import deepcopy
c = deepcopy(a)
c

[10, 20, [[...], 30]]


[10, 20, [[...], 30]]

깊은 복사는 일반적인 상황이 아님미다! 객체에는 순환 참조가 있을 수도 있는데 이러면 아주 무한 루프돌고 큰일 나거든요.. 깊은 복사는 이런 것(복사 안해도 될 거까지)까지 정말! 잘! 복사하는 것을 볼 수 있습니다. 그리고 어떤 경우에서는 깊은 복사가 너무 깊어서 문제가 된다네요.. 외부 리소스나 싱글톤은 복사되면 안되기 때문에 `copy()`와 `deepcopy()`를 `__copy()__`와 `__deepcopy()__`로 적절히 컨트롤해서 구현할 수 있습니다.

## 8.4 참조로서의 함수 매개변수
파이썬에서 매개변수를 전달하는 방법은 'call by sharing'밖에 없다.  
**Call by sharing?** 함수의 각 형식 매개 변수가 인수의 각 참조 사본을 가져온다. 다시 말하면 함수의 매개변수는 실제 인수들의 별명(alias)이 된다는 말이다! 매개변수에 어떤 자료형을 넣느냐에 따라 결과가 달라진다.

In [12]:
#Example 8-11
def f(a, b):
    a += b
    return a

x = 1
y = 2
print(f(x, y))
print(x, y)

a = [1, 2]
b = [3, 4]
print(f(a, b))
print(a, b)

t = (10, 20)
u = (30, 40)
print(f(t, u))
print(t, u)

3
1 2
[1, 2, 3, 4]
[1, 2, 3, 4] [3, 4]
(10, 20, 30, 40)
(10, 20) (30, 40)


### Mutable type을 매개변수로 사용: 나쁜 아이디어

In [13]:
#Example 8-12
class HauntedBus:
    """A bus model haunted by ghost passengers"""
    
    def __init__(self, passengers=[]):
        self.passengers = passengers
        
    def pick(self, name):
        self.passengers.append(name)
    
    def drop(self, name):
        self.passengers.remove(name)

In [14]:
#Example 8-13
bus1 = HauntedBus(['Alice', 'Bill'])
print(bus1.passengers)
bus1.pick('Charlie')
bus1.drop('Alice')
print(bus1.passengers)
bus2 = HauntedBus()
bus2.pick('Carrie')
print(bus2.passengers)
bus3 = HauntedBus()
print(bus3.passengers)
bus3.pick('Dave')
print(bus2.passengers)
print(bus2.passengers is bus3.passengers)
print(bus1.passengers)

['Alice', 'Bill']
['Bill', 'Charlie']
['Carrie']
['Carrie']
['Carrie', 'Dave']
True
['Bill', 'Charlie']


문제는 `passengers`가 비어있을 때만 발생한다.(bus1은 문제가 없었음) 이런 일은 `self.passengers`가 기본값(==[])에 대한 alias가 되기 때문이다. 여기서 문제는 각각의 기본값은 함수가 정의될 때 평가되고 함수 객체의 속성이 되어버린다. 그렇기 때문에 기본값이 mutable 객체고 이걸 바꾸면 미래에 영향을 미치게 되는 것이다.

In [15]:
print(dir(HauntedBus.__init__))
print(HauntedBus.__init__.__defaults__)

print(HauntedBus.__init__.__defaults__[0] is bus2.passengers)

['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']
(['Carrie', 'Dave'],)
True


이러한 이유 때문에, mutable 객체를 매개변수로 받아야 하는 경우 기본값을 `None`으로 받는 것이다. 

### Mutable 매개변수와 방어적인 프로그래밍
~~(방어운전 같은건가)~~  
mutable 매개변수를 사용해서 코딩할 때, 호출자가 인수가 변할 것을 기대하는지를 고려해야 한다.

In [16]:
#Example 8-15
class TwilightBus:
    """A bus model that makes passengers vanish"""
    
    def __init__(self, passengers=None):
        if passengers is None:
            self.passnegers = []
        else:
            self.passengers = passengers
            
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)

In [17]:
#Example 8-14
basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)
bus.drop('Tina')
bus.drop('Pat')
basketball_team

['Sue', 'Maya', 'Diana']

~~전 그저 버스에서 내렸을 뿐인데..~~

문제는 `__init__()`에 있다. `passengers`를 받을 때 `list(passengers)`로 받아서 넣어주면 복사본을 넣어주기 때문에 매개변수에 영향을 미치지 않는다. 또 저렇게 받으면 어떠한 mutable 객체도 받을 수 있어서 좋다.

## 8.5 del과 가비지 컬렉션
<blockquote>Objects are never explicitly destroyed; however, when they become unreachable they may be garbage-collected.</blockquote>

`del`문은 객체를 지우는 것이 아니라 이름을 지운다. *An object may be garbage collected as result of a del command, but only if the variable deleted holds the last reference to the object, or if the object becomes unreachable.*

`__del__`과는 또 다르다! 구분할 것.

CPython에서 가비지 컬렉션의 주 알고리즘은 reference counting이다. `__del__()`을 호출하면 메모리가 해제된다. refcount가 0이 되면 즉시 파괴된다. CPython 2.0에서 순환 참조 그룹을 감지하기 위한 세대(?) 가비지 컬렉션 알고리즘을 추가했다. 

하지만 Python은 더 복잡한 알고리즘을 사용하고 참조가 없어도 즉시 `__del__()`이 호출되지는 않는다.

In [18]:
import weakref #선 넘네
s1 = {1, 2, 3}
s2 = s1

def bye():
    print('Gone with the wind...')
    
ender = weakref.finalize(s1, bye)
print(ender.alive)
del s1
print(ender.alive)
s2 = 'spam' #아까 말했던 rebinding이 이런 상황
print(ender.alive)


True
True
Gone with the wind...
False
